In [1]:
import praw
import pandas as pd

In [2]:
reddit = praw.Reddit(
    client_id="y25BJERt0d-YtQKuTk7gUQ",
    client_secret="fBb7prfE4DcIj2Cv4l7Dw73ziLZFyQ",
    user_agent="sl-nlp-project:v1 (by /u/SpareUse7114)", 
)
reddit.read_only

True

In [3]:
subreddit = reddit.subreddit("srilanka")
subreddit

Subreddit(display_name='srilanka')

## Gather text posts

In [4]:
sources = [
    subreddit.hot(limit=None),
    subreddit.new(limit=None),
    subreddit.top(time_filter="all", limit=None),
    subreddit.controversial(time_filter="all", limit=None),
    subreddit.rising(limit=None),
    subreddit.random_rising(limit=None)
]


### Posts

In [5]:
posts_data = []

try:
    for source in sources:
        for submission in source:
            if submission.selftext.strip():
                posts_data.append({
                    "id": submission.id,
                    "post_title": submission.title,
                    "type": "post",
                    "body": submission.selftext,
                    "score": submission.score,
                    "url": submission.url,
                    "created_utc": submission.created_utc
                })
        print(f"Collected {len(posts_data)} posts...")

except Exception as e:
    print(f"Error collecting posts: {e}")


Collected 958 posts...
Collected 1909 posts...
Collected 2011 posts...
Collected 2304 posts...
Collected 2328 posts...
Collected 2352 posts...


In [6]:
df_text_posts = pd.DataFrame(posts_data)
df_text_posts.head()

,id,post_title,type,body,score,url,created_utc
0,1osnjtw,Here we go. People opposing the implementation...,post,**\*Above text is translated from Tamil to Eng...,77,https://i.redd.it/prqk8cgf990g1.png,1.762705e+09
1,1oselyv,Does ragging like this still exist?,post,Does ragging like this still exist for girls i...,108,https://i.redd.it/itaviwni470g1.png,1.762679e+09
2,1oshqdu,Ahangama and Tuktuk Mafia,post,Hello guys \n\nI am a Tourist from Germany and...,58,https://www.reddit.com/r/srilanka/comments/1os...,1.762690e+09
3,1oso9zm,I feel like Sri Lanka will never honestly chan...,post,"Genuine question, I know that we have a decent...",17,https://www.reddit.com/r/srilanka/comments/1os...,1.762707e+09
4,1osmi3z,Trivializing Sri Lankan food by calling most s...,post,"As the title says, people call every dish they...",15,https://www.reddit.com/r/srilanka/comments/1os...,1.762703e+09


In [7]:
df_text_posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           2352 non-null   object 
 1   post_title   2352 non-null   object 
 2   type         2352 non-null   object 
 3   body         2352 non-null   object 
 4   score        2352 non-null   int64  
 5   url          2352 non-null   object 
 6   created_utc  2352 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 128.8+ KB


In [8]:
if not df_text_posts.empty:
    df_text_posts.to_csv("raw data\srilanka_posts_text.csv", index=False, encoding="utf-8")
    print("Data collection complete. Saved to raw data\srilanka_posts_text.csv")
    posts_data.clear()
else:
    print("\nNo text posts were collected.")

Data collection complete. Saved to raw data\srilanka_posts_text.csv


### Comments

In [9]:
comments_data = []

try:
    for post_id in df_text_posts["id"]:
        submission = reddit.submission(id=post_id)

        # Expand all comments (up to reasonable depth)
        submission.comments.replace_more(limit=20)

        for comment in submission.comments.list():
            comments_data.append({
                "id": post_id,
                "post_title": None,
                "type": "comment",
                "body": comment.body,
                "score": comment.score,
                "url": comment.permalink,
                "created_utc": comment.created_utc
            })
        print(f"Collected {len(comments_data)} comments...")

except Exception as e:
    print(f"Error collecting comments: {e}")


Collected 29 comments...
Collected 99 comments...
Collected 124 comments...
Collected 144 comments...
Collected 145 comments...
Collected 167 comments...
Collected 171 comments...
Collected 184 comments...
Collected 194 comments...
Collected 212 comments...
Collected 213 comments...
Collected 215 comments...
Collected 219 comments...
Collected 220 comments...
Collected 227 comments...
Collected 287 comments...
Collected 289 comments...
Collected 290 comments...
Collected 298 comments...
Collected 305 comments...
Collected 310 comments...
Collected 327 comments...
Collected 331 comments...
Collected 332 comments...
Collected 359 comments...
Collected 371 comments...
Collected 372 comments...
Collected 434 comments...
Collected 443 comments...
Collected 502 comments...
Collected 509 comments...
Collected 511 comments...
Collected 512 comments...
Collected 516 comments...
Collected 516 comments...
Collected 522 comments...
Collected 525 comments...
Collected 526 comments...
Collected 552 

In [10]:
df_text_comments = pd.DataFrame(comments_data)
df_text_comments.head()

,id,post_title,type,body,score,url,created_utc
0,1osnjtw,None,comment,By now my default position on any given topic ...,29,/r/srilanka/comments/1osnjtw/here_we_go_people...,1.762706e+09
1,1osnjtw,None,comment,A priest lecturing about sex ed is wild. Bro’s...,43,/r/srilanka/comments/1osnjtw/here_we_go_people...,1.762706e+09
2,1osnjtw,None,comment,Ew wtf is this 😂🤌🏻,24,/r/srilanka/comments/1osnjtw/here_we_go_people...,1.762706e+09
3,1osnjtw,None,comment,The Cardinal of the organization with the larg...,20,/r/srilanka/comments/1osnjtw/here_we_go_people...,1.762707e+09
4,1osnjtw,None,comment,The cardinal is just jealous because he’s supp...,15,/r/srilanka/comments/1osnjtw/here_we_go_people...,1.762708e+09


In [11]:
df_text_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49415 entries, 0 to 49414
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           49415 non-null  object 
 1   post_title   0 non-null      object 
 2   type         49415 non-null  object 
 3   body         49415 non-null  object 
 4   score        49415 non-null  int64  
 5   url          49415 non-null  object 
 6   created_utc  49415 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 2.6+ MB


In [12]:
if not df_text_comments.empty:
    df_text_comments.to_csv("raw data\srilanka_comments_text.csv", index=False, encoding="utf-8")
    print("Data collection complete. Saved to raw data\srilanka_comments_text.csv")
    comments_data.clear()
else:
    print("\nNo text comments were collected.")

Data collection complete. Saved to raw data\srilanka_comments_text.csv


## Gather non-text posts

In [13]:
sources = [
    subreddit.hot(limit=None),
    subreddit.new(limit=None),
    subreddit.top(time_filter="all", limit=None),
    subreddit.controversial(time_filter="all", limit=None),
    subreddit.rising(limit=None),
    subreddit.random_rising(limit=None)
]

### Posts

In [14]:
try:
    for source in sources:
        for submission in source:
            if not submission.selftext.strip():
                post_hint = getattr(submission, "post_hint", None)
                is_gallery = getattr(submission, "is_gallery", False)

                if is_gallery:
                    post_hint = "gallery"
                    
                posts_data.append({
                    "id": submission.id,
                    "post_title": submission.title,
                    "type": "post",
                    "body": None,
                    "score": submission.score,
                    "url": submission.url,
                    "post_hint": post_hint,
                    "created_utc": submission.created_utc
                })
        print(f"Collected {len(posts_data)} posts...")

except Exception as e:
    print(f"Error: {e}")

Collected 42 posts...
Collected 81 posts...
Collected 229 posts...
Collected 295 posts...
Collected 296 posts...
Collected 297 posts...


In [15]:
df_non_text_posts = pd.DataFrame(posts_data)
df_non_text_posts.head()

,id,post_title,type,body,score,url,post_hint,created_utc
0,1osg71i,Hey so I'm we've noticed you enjoying our data...,post,None,19,https://i.redd.it/hdoa7id8m70g1.jpeg,image,1.762685e+09
1,1orfa1r,Life for Stray Dogs - what do you think about ...,post,None,288,https://i.redd.it/alcceyjxfyzf1.jpeg,image,1.762573e+09
2,1ornheo,Scam alert-I don’t even have an hsbc credit ca...,post,None,33,https://i.redd.it/je6sd3vdu00g1.jpeg,image,1.762603e+09
3,1oqx502,Sri Lankan mobile network providers,post,None,209,https://i.redd.it/rjcs1y5tquzf1.jpeg,image,1.762529e+09
4,1oqsdi8,Where i can buy something like this?,post,None,82,https://i.redd.it/kf0u9gf0qtzf1.png,image,1.762516e+09


In [16]:
df_non_text_posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           297 non-null    object 
 1   post_title   297 non-null    object 
 2   type         297 non-null    object 
 3   body         0 non-null      object 
 4   score        297 non-null    int64  
 5   url          297 non-null    object 
 6   post_hint    288 non-null    object 
 7   created_utc  297 non-null    float64
dtypes: float64(1), int64(1), object(6)
memory usage: 18.7+ KB


In [17]:
import pytesseract, requests
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
from PIL import Image
from io import BytesIO
from newspaper import Article

In [18]:
def convert_to_text(url, post_hint, is_gallery=False, media_metadata=None):
    """
    Convert a Reddit submission to text depending on its post type.
    Handles images, galleries, links, and videos.
    """
    text_output = None

    try:
        if post_hint == "image":
            # Single image OCR
            headers = {"User-Agent": "Mozilla/5.0"}
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            img = Image.open(BytesIO(response.content))
            text_output = pytesseract.image_to_string(img) or None

        elif post_hint == "link":
            # Extract text from linked article
            article = Article(url, browser_user_agent="Mozilla/5.0")
            article.download()
            article.parse()
            text_output = article.text.strip() or None

        elif post_hint in ["hosted:video", "rich:video"] or "video" in str(post_hint):
            # Video posts (no OCR)
            text_output = None

        elif is_gallery and media_metadata:
            # Loop through gallery images and extract text
            gallery_texts = []
            for media_id, media_info in media_metadata.items():
                img_url = media_info["s"]["u"].replace("&amp;", "&")
                response = requests.get(img_url, timeout=10)
                img = Image.open(BytesIO(response.content))
                text = pytesseract.image_to_string(img)
                if text.strip():
                    gallery_texts.append(text.strip())

            text_output = ". ".join(gallery_texts) or None

        else:
            text_output = None

    except Exception as e:
        print(f"Error converting {url}: {e}")
        text_output = None

    return text_output

In [19]:
df_non_text_posts["body"] = df_non_text_posts.apply(
    lambda row: convert_to_text(
        row["url"],
        row["post_hint"],
        getattr(row, "is_gallery", False),
        getattr(row, "media_metadata", None)
    ),
    axis=1
)


In [20]:
df_non_text_posts = df_non_text_posts.drop(columns=["post_hint"])

In [21]:
df_non_text_posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           297 non-null    object 
 1   post_title   297 non-null    object 
 2   type         297 non-null    object 
 3   body         147 non-null    object 
 4   score        297 non-null    int64  
 5   url          297 non-null    object 
 6   created_utc  297 non-null    float64
dtypes: float64(1), int64(1), object(5)
memory usage: 16.4+ KB


In [22]:
if not df_non_text_posts.empty:
    df_non_text_posts.to_csv("raw data\srilanka_posts_non_text.csv", index=False, encoding="utf-8")
    print("Data collection complete. Saved to raw data\srilanka_posts_non_text.csv")
    posts_data.clear()
else:
    print("\nNo text posts were collected.")

Data collection complete. Saved to raw data\srilanka_posts_non_text.csv


### Comments

In [29]:
try:
    for post_id in df_non_text_posts["id"]:
        submission = reddit.submission(id=post_id)

        # Expand all comments (up to reasonable depth)
        submission.comments.replace_more(limit=20)

        for comment in submission.comments.list():
            comments_data.append({
                "id": post_id,
                "post_title": None,
                "type": "comment",
                "body": comment.body,
                "score": comment.score,
                "url": comment.permalink,
                "created_utc": comment.created_utc
            })
        print(f"Collected {len(comments_data)} comments...")

except Exception as e:
    print(f"Error collecting comments: {e}")

Collected 10 comments...
Collected 61 comments...
Collected 64 comments...
Collected 80 comments...
Collected 111 comments...
Collected 126 comments...
Collected 132 comments...
Collected 159 comments...
Collected 186 comments...
Collected 421 comments...
Collected 449 comments...
Collected 453 comments...
Collected 512 comments...
Collected 568 comments...
Collected 571 comments...
Collected 671 comments...
Collected 712 comments...
Collected 714 comments...
Collected 723 comments...
Collected 723 comments...
Collected 726 comments...
Collected 791 comments...
Collected 827 comments...
Collected 864 comments...
Collected 995 comments...
Collected 1064 comments...
Collected 1067 comments...
Collected 1080 comments...
Collected 1100 comments...
Collected 1109 comments...
Collected 1133 comments...
Collected 1137 comments...
Collected 1150 comments...
Collected 1156 comments...
Collected 1165 comments...
Collected 1166 comments...
Collected 1191 comments...
Collected 1193 comments...
Col

In [30]:
df_non_text_comments = pd.DataFrame(comments_data)
df_non_text_comments.head()

,id,post_title,type,body,score,url,created_utc
0,1osg71i,None,comment,**Please note that this is a meme/satire threa...,1,/r/srilanka/comments/1osg71i/hey_so_im_weve_no...,1.762685e+09
1,1osg71i,None,comment,Greedy buggers,9,/r/srilanka/comments/1osg71i/hey_so_im_weve_no...,1.762687e+09
2,1osg71i,None,comment,How dare you enjoy something!,5,/r/srilanka/comments/1osg71i/hey_so_im_weve_no...,1.762705e+09
3,1osg71i,None,comment,um*,1,/r/srilanka/comments/1osg71i/hey_so_im_weve_no...,1.762685e+09
4,1osg71i,None,comment,Lol how ironic! I got the same message today 🤣,1,/r/srilanka/comments/1osg71i/hey_so_im_weve_no...,1.762715e+09


In [31]:
df_non_text_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13870 entries, 0 to 13869
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           13870 non-null  object 
 1   post_title   0 non-null      object 
 2   type         13870 non-null  object 
 3   body         13870 non-null  object 
 4   score        13870 non-null  int64  
 5   url          13870 non-null  object 
 6   created_utc  13870 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 758.6+ KB


In [32]:
if not df_non_text_comments.empty:
    df_non_text_comments.to_csv("raw data\srilanka_comments_non_text.csv", index=False, encoding="utf-8")
    print("Data collection complete. Saved to raw data\srilanka_comments_non_text.csv")
    comments_data.clear()
else:
    print("\nNo text comments were collected.")

Data collection complete. Saved to raw data\srilanka_comments_non_text.csv


In [33]:
combined_df = pd.concat([df_text_posts, df_text_comments, df_non_text_posts, df_non_text_comments], ignore_index=True)

In [35]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65934 entries, 0 to 65933
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           65934 non-null  object 
 1   post_title   2649 non-null   object 
 2   type         65934 non-null  object 
 3   body         65784 non-null  object 
 4   score        65934 non-null  int64  
 5   url          65934 non-null  object 
 6   created_utc  65934 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 3.5+ MB


In [ ]:
if not combined_df.empty:
    combined_df.to_csv("raw data\srilanka_all_text.csv", index=False, encoding="utf-8")
    print("Saved to raw data\srilanka_all_text.csv")
else:
    print("\nNo data.")

Saved to raw data\srilanka_all_text.csv.csv
